In [3]:
import importlib

import copy
from tabulate import tabulate

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MySimpleLinearRegressor, MyNaiveBayesClassifier, MyDecisionTreeClassifier, MyRandomForestClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

In [15]:
wildfire_fname = os.path.join("data","clean_fire_data.csv")
wildfire_table = MyPyTable()
wildfire_table.load_from_file(wildfire_fname)

fire_date = wildfire_table.get_column("date")
county = wildfire_table.get_column("county")
acres = wildfire_table.get_column("acres")
cause = wildfire_table.get_column("cause")
lat = wildfire_table.get_column("lat")
lng = wildfire_table.get_column("lon")
bin_lat = wildfire_table.get_column("binlat")
bin_lon = wildfire_table.get_column("binlon")

wildfire_X = [[fire_date[i], county[i], cause[i], bin_lat[i], bin_lon[i]] for i in range(len(fire_date))]
wildfire_y = [int(x) for x in acres]





In [16]:
print(len(wildfire_X))
wildfire_X_new = []
wildfire_y_new = []
for i, x in enumerate(wildfire_X):
    add_instance = True
    if x[1] == "None" or x[1] == "NO DATA":
        add_instance = False
    elif x[2] == "None" or x[2] == "Logging" or x[2] == "Miscellaneou" or x[2] == "Under Invest" or x[2] == "Undetermined" or x[2] == "Railroad" or x[2] == "Arson":
        add_instance = False
    elif wildfire_y[i] < 10:
        add_instance = False
    # print(wildfire_y[i], add_instance)
    if add_instance:
        # print("adding", wildfire_y_new)
        wildfire_X_new.append(x)
        wildfire_y_new.append(wildfire_y[i])

# print(wildfire_y_new)
print(len(wildfire_X_new))
wildfire_X = wildfire_X_new
wildfire_y = wildfire_y_new

58945
1357


In [19]:
X_train, X_test, y_train, y_test = myevaluation.train_test_split(copy.deepcopy(wildfire_X), copy.deepcopy(wildfire_y), test_size=100)


In [21]:

max_accuracy = 0
for F in range(1, 6):
    for N in range(1, 30):
        for M in range(1, N+1):
            rf = MyRandomForestClassifier()
            rf.fit(wildfire_X, wildfire_y, F=F, N=N, M=M)

            predictions = []
            for i, x in enumerate(X_test):
                prediction = rf.predict([x])
                # print(prediction, y_test[i])
                # print(prediction)
                predictions.append(int(prediction[0] == y_test[i]))
                if sum(predictions)/len(predictions) > max_accuracy:
                    print("F:", F, "N:", N, "M:", M, "Accuracy:", sum(predictions)/len(predictions))
                    max_accuracy = sum(predictions)/len(predictions)
    print("done")

F: 1 N: 1 M: 1 Accuracy: 0.07142857142857142
F: 1 N: 1 M: 1 Accuracy: 0.08333333333333333
F: 1 N: 1 M: 1 Accuracy: 0.09090909090909091
F: 1 N: 2 M: 1 Accuracy: 0.1
F: 1 N: 2 M: 1 Accuracy: 0.12
F: 1 N: 3 M: 1 Accuracy: 0.2
F: 1 N: 7 M: 1 Accuracy: 0.25
F: 1 N: 7 M: 3 Accuracy: 1.0


KeyboardInterrupt: 

In [17]:
test_size = 10

X_train, X_test, y_train, y_test = myevaluation.train_test_split(copy.deepcopy(wildfire_X), copy.deepcopy(wildfire_y), test_size=test_size, shuffle=True)

nb = MyKNeighborsClassifier()
nb.fit(X_train, y_train)
predictions = []
for i, x in enumerate(X_test):
    sys.stdout.write("\r" + str(i) + "/" + str(len(X_test) -1) + "    ")
    sys.stdout.flush()
    prediction = nb.predict([x])
    predictions.append(prediction[0])

print()
acc = round(sum([int(x==y) for x,y in zip(predictions, y_test)])/len(predictions), 2)
print("KNN: accuracy = " + str(acc) + " error rate = " + str(1-acc))

9/9    
KNN: accuracy = 0.2 error rate = 0.8


In [18]:
print("getting training folds...")
X_train_folds, X_test_folds = myevaluation.stratified_kfold_cross_validation(wildfire_X, wildfire_y, n_splits=10)

getting training folds...


In [19]:
# k-fold stratified cross validation


print()
print("Stratified 10-Fold Cross Validation")

# decision trees for stratified k-fold
nb_predicted = []
nb_actual = []
# only does the first for now since it takes so long
for i in range(0, 1):
    X_train, X_test, y_train, y_test = myutils.get_trains_and_tests(wildfire_X, wildfire_y, X_train_folds[i], X_test_folds[i])
    nb = MyKNeighborsClassifier()
    nb.fit(X_train, y_train)
    predictions = []
    for i, x in enumerate(X_test):
        sys.stdout.write("\r" + str(i) + "/" + str(len(X_test) -1) + "    ")
        sys.stdout.flush()
        prediction = nb.predict([x])
        predictions.append(prediction[0])

    print()
    predicted = [ x for x in predictions]
    actual = [x for x in y_test]
    
    nb_predicted +=  predicted
    nb_actual += actual

acc = round(sum([int(x==y) for x,y in zip(predicted, actual)])/len(predicted), 2)
print("Naive Bayes: accuracy = " + str(acc) + " error rate = " + str(1-acc))


Stratified 10-Fold Cross Validation
4220/4220    
Naive Bayes: accuracy = 0.28 error rate = 0.72
